In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import sys
from os import listdir

In [54]:
def create_paper_summary_data_frame():
    doi = input("DOI for paper: ")
    human_src_id = input("Human Src ID: ")
    directory = input("Directory for paper: ")
    
    #list of all the [folders/plots] for paper ()
    plots_in_dir = !ls $directory 
    num_plots = len(plots_in_dir)
    
    for i in range(num_plots):
        paper_descript = input("Describe some key aspects about the paper ")
        plot_foldername = directory + "/" + plots_in_dir[i]
        
        # list of all the csv's associated with plot
        data_segs_for_plot = !ls $plot_foldername 
        print("This is plot number " + str(i))
        num_segs_in_plot = len(data_segs_for_plot)
        

        
        cols = ['Air Vel [cm/s]','FSR [mm/s]','d_c [mm]', 'd_i [mm]',
                      'P_atm', '[O2]', 'g', 'q_ext [kW/m^2]', 'core material', 
                      'insulation material','orientation (vert/horiz)', 'up | down', 'Human Src ID', 'DOI', 'Plot #']
        final_df = pd.DataFrame(columns = cols)
        x_value, y_value = 0, 1
        for j in range(1):#num_segs_in_plot):
            name_of_seg_in_plot = plot_foldername + "/" + data_segs_for_plot[j]
            print(name_of_seg_in_plot)
            skeleton_row = []
            for k in cols:
                if k == 'Human Src ID':
                    skeleton_row.append(human_src_id)
                    continue
                if k == 'DOI':
                    skeleton_row.append(doi)
                    continue
                if k == 'Plot #':
                    skeleton_row.append(i)
                    continue
                val = input("For data seg " + data_segs_for_plot[j] + " " + k + " ")
                if (str(val) == 'x'):
                    x_value = cols.index(k)
                    skeleton_row.append(0)
                elif (str(val) == 'y'):
                    y_value = cols.index(k)
                    skeleton_row.append(0)
                elif k in ['core material', 'insulation material', 'orientation (vert/horiz)', 'up | down']:
                    skeleton_row.append(str(val or 'NA'))
                else: 
                    skeleton_row.append(float(val or 'nan'))
            csv_data = pd.read_csv(name_of_seg_in_plot, skiprows=5, index_col=False)
            skeleton_df = pd.DataFrame(np.array([skeleton_row]), columns=cols)
            skeleton_df = pd.concat([skeleton_df]*csv_data.shape[0], ignore_index=True)
            csv_data.columns = [0,1]
            skeleton_df[cols[x_value]], skeleton_df[cols[y_value]] = csv_data[0], csv_data[1]
            final_df = final_df.append(skeleton_df, ignore_index=True)
            final_df
        return final_df

In [55]:
data = create_paper_summary_data_frame()


DOI for paper: 1231234123123
Human Src ID: Urban 2019
Directory for paper: kobayashi
Describe some key aspects about the paper d
This is plot number 0
kobayashi/1/8mm_Cu.csv
For data seg 8mm_Cu.csv Air Vel [cm/s] 
For data seg 8mm_Cu.csv FSR [mm/s] 
For data seg 8mm_Cu.csv d_c [mm] 313
For data seg 8mm_Cu.csv d_i [mm] 200
For data seg 8mm_Cu.csv P_atm 100
For data seg 8mm_Cu.csv [O2] 
For data seg 8mm_Cu.csv g 
For data seg 8mm_Cu.csv q_ext [kW/m^2] 
For data seg 8mm_Cu.csv core material 
For data seg 8mm_Cu.csv insulation material 
For data seg 8mm_Cu.csv orientation (vert/horiz) 
For data seg 8mm_Cu.csv up | down 


In [56]:
def create_scatter_from_df(dframe):
    

,Air Vel [cm/s],FSR [mm/s],d_c [mm],d_i [mm],P_atm,[O2],g,q_ext [kW/m^2],core material,insulation material,orientation (vert/horiz),up | down,Human Src ID,DOI,Plot #
0,0.134322,0.522128,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
1,14.111400,0.450124,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
2,23.839900,0.425121,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
3,31.571900,0.409509,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
4,41.551700,0.375101,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
5,54.019200,0.365802,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
6,65.975200,0.414513,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
7,75.700900,0.402054,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
8,99.881300,0.413240,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
9,125.329000,0.331929,313.0,200.0,100.0,nan,nan,nan,NA,NA,NA,NA,Urban 2019,1231234123123,0
